In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [2]:
# 1. .env 환경변수 불러오기

load_dotenv()

WATSONX_API = os.environ['API_KEY']
PROJECT_ID = os.environ['PROJECT_ID']
IBM_URL = os.environ['IBM_CLOUD_URL']


In [3]:
# 2. 데이터 로드 (정상 + 스미싱)

df1 = pd.read_csv("labeled_normal_messages.csv")    # 정상
df2 = pd.read_csv("labeled_smishing_messages.csv")  # 스미싱
df = pd.concat([df1, df2], ignore_index=True)

In [4]:
# 3. 문서 준비
texts = df["message"].tolist()
metadatas = [{"label": label} for label in df["label"].tolist()]

In [5]:
# 4. 텍스트 청크 분할

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=80, # 평균적인 문자메시지 길이 고려
    chunk_overlap=10,
    length_function=len
)

docs = text_splitter.create_documents(texts, metadatas=metadatas)

In [6]:
# 5. 임베딩 모델 설정

# MODEL_ID = 'ibm/slate-125m-english-rtrvr'
MODEL_ID = 'ibm/granite-embedding-278m-multilingual' # English -> Multilingual
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3, # 입력 텍스트가 모델 최대 토큰 수를 초과할 경우 가장 뒤쪽을 자른다는 의미
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

watsonx_embedding = WatsonxEmbeddings(
    model_id=MODEL_ID,
    url=IBM_URL,
    project_id=PROJECT_ID,
    params=embed_params,
    apikey=WATSONX_API
)

In [7]:
# 6. ChromaDB 설정 및 저장

persist_directory = "./chroma_store" # Local에 저장할 경로

vectorstore = Chroma(
    collection_name="sms_messages",
    embedding_function=watsonx_embedding,
    persist_directory=persist_directory
)

vectorstore.add_documents(docs)
# vectorstore.persist()

print("✅ 임베딩 및 Vector DB 저장 완료!")

✅ 임베딩 및 Vector DB 저장 완료!


In [10]:
# 7. 유사 문장 검색 테스트

query = "[Web발신] 모바일 인증번호가 도착했습니다. 링크 접속 필요 http://short.kr/gbkg8"
results = vectorstore.similarity_search_with_score(query, k=3) # 코사인 유사도 기반

print("\n🔍 유사한 메시지 Top 3:")
for i, (doc, score) in enumerate(results):
    print(f"\n[{i+1}] ({doc.metadata['label']}, score={score:.4f})")
    print(doc.page_content)



🔍 유사한 메시지 Top 3:

[1] (스미싱, score=0.0000)
[Web발신] 과태료 미납 내역 발생. 자세한 정보 확인 바랍니다. http://t.ly/fl2j2l23c

[2] (스미싱, score=0.0000)
[Web발신] [교통위반] 범칙금 고지서 발송 완료. 확인 필수 http://t.ly/ewzu3

[3] (스미싱, score=0.0000)
[Web발신] 모바일 돌잔치 초대장을 보내드립니다. 확인해주세요. http://me2.do/pfe230f
